### Libraries

In [23]:
import torch
from torchvision import transforms
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import numpy as np
import requests
import os
import time
import faiss
import json

In [24]:
images_folder_path = "../../1) Data_Collection/OID/Dataset/train"
output = "dinov2_embeddings"

bin_file = f'{output}/dinoIndex.bin'

In [25]:
index = faiss.IndexFlatL2(768)

In [26]:
# Load DINOv2 model (replace with the actual loading function or model URL)
def load_dinov2_model():
    model = AutoModel.from_pretrained("facebook/dinov2-base")
    model.eval()  # Set model to evaluation mode
    return model

# Preprocess the image
def preprocess_image(image):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    return transform(image).unsqueeze(0)  # Add batch dimension

# Generate embeddings
def generate_embeddings(model, image_tensor):
    with torch.no_grad():
        embeddings = model(image_tensor)

    return embeddings["last_hidden_state"].cpu().numpy().mean(axis=1)

# Save embeddings to a .bin file
def save_embeddings_to_bin(file_path):
    faiss.write_index(index, file_path)

In [27]:
# Main function
def main(image_path, output_path):
    
    global total_time, total_images

    model = load_dinov2_model()

    start_time = time.time()

    image_tensor = preprocess_image(image_path)
    embeddings = generate_embeddings(model, image_tensor)

    end_time = time.time()
    elapsed_time = end_time - start_time

    total_time += elapsed_time
    total_images += 1

    index.add(embeddings)

In [28]:
total_time = 0
total_images = 0

In [29]:
indexes = {}
idx = 0

for one_category in os.listdir(images_folder_path):
    one_category_path = os.path.join(images_folder_path, one_category)

    all_images_in_category_folder = [x for x in os.listdir(one_category_path) if x.endswith("jpg")]

    for img in all_images_in_category_folder:

        img_path = os.path.join(one_category_path, img)

        image = Image.open(img_path)
        image = image.convert("RGB")
        
        output_path = os.path.join(output, img[:-3]+"bin")

        main(image, output_path)

        indexes[idx] = img_path
        idx += 1

save_embeddings_to_bin(bin_file)

In [30]:
average_time = total_time / total_images if total_images > 0 else 0

print(f"Total Time: {total_time:.2f} seconds")
print(f"Average Time per Image: {average_time:.2f} seconds")

Total Time: 16.30 seconds
Average Time per Image: 0.30 seconds


### Saving Indices

In [31]:
with open(f'{output}/indices.json', 'w') as file:
    json.dump(indexes, file, indent=4)